In [4]:
!sudo apt update
!sudo apt-get update
!sudo apt-get -y install nasm

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-d

In [ ]:
%%file grayscaling.c
extern void GS_asm(float *color, float *salida, int limite);

void GC(float *color, float *salida, int limite)
{
    int j=0;
    for(int i=0;i<limite;i++)
    {
        salida[i]=color[j]*0.2125+color[j+1]*0.7174+color[j+2]*0.0721;
        j=j+3;
    }
}
//convertir en flatten 

Overwriting grayscaling.c


In [ ]:
%%file grayscale_asm.asm
; este codigo contiene la informacion en asm

    global GS_
    section .text
GS_:
    ; [rdi] es color
    ; [rsi] es salida
    ; rcx es el limite

    xorpd xmm0, xmm0; color[0]
    xorpd xmm1, xmm1; color[1]
    xorpd xmm2, xmm2; color[2]
    xorpd xmm3, xmm3; salida[]

    mov r8,rdx  ; rdx va a poner el contador a r8
    cmp rdx, 0
    je done

constantes:
    xorpd xmm4, xmm4
    xorpd xmm5, xmm5
    xorpd xmm6, xmm6
    xorpd xmm7, xmm7

    mov r10, 2125 
    mov r11, 10000
    cvtsi2ss xmm4, r10
    cvtsi2ss xmm5, r11
    divss xmm4, xmm5

    mov r10, 7174
    mov r11, 10000
    cvtsi2ss xmm6, r10
    cvtsi2ss xmm5, r11
    divss xmm6, xmm5

    mov r10, 721
    mov r11, 10000
    cvtsi2ss xmm7, r10
    cvtsi2ss xmm5, r11
    divss xmm7, xmm5
    
    

loop1: 
    ;el valor de rdi va a almacenarse en cada xmmX cada 3 veces
    ;con un incremento de 12 variables
    ; 0.2125
    ; color[0]
    movss xmm0, [rdi] 
    mulss xmm0, xmm4
    add rdi, 4
    ; 0.7174
    ; color[1] 
    movss xmm1, [rdi] 
    mulss xmm1, xmm6
    add rdi, 4
    ; 0.0721
    ; color[2]
    movsd xmm2, [rdi] 
    mulss xmm2, xmm7
    add rdi, 4
    
    addss xmm3,xmm0
    addss xmm3,xmm1
    addss xmm3,xmm2

    movss [rsi], xmm3
    add rsi, 4
    
    xorpd xmm3, xmm3;
    ;xorpd xmm0, xmm0;
    ;xorpd xmm1, xmm1;
    ;xorpd xmm2, xmm2;

    sub r8,1      ; decrement
    cmp r8,0
    jne loop1   ; Loop while less or equal
done:
    ret

Overwriting grayscale_asm.asm


In [ ]:
! gcc -shared grayscaling.c -o GC_c.so

In [ ]:
! nasm -f elf64 grayscale_asm.asm -o grayscale_asm_.o
! gcc -shared grayscale_asm_.o grayscaling.c -o grayscale_ASMO.so


In [ ]:
pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from PIL import Image
import numpy as np
import ctypes
import time
from os import scandir, getcwd
import matplotlib.pyplot as plt
import statistics
import cv2

#SE USA ESTA FUNCION PARA OBTENER EL NOMBRE DE LOS ARCHIVOS DE UNA CARPETA
def ls(ruta = getcwd()):
    return [arch.name for arch in scandir(ruta) if arch.is_file()]
 
OKBLUE = '\033[94m'
OKCYAN = '\033[96m'
OKGREEN = '\033[92m'
BOLD = '\033[1m'
YELLOW='\033[93m'
purple='\033[35m'



#SE UTILIZARAN IMAGENES DE FORMATO JPEG POR UNA MAYOR RESOLUCION
#grayscl_asm = ctypes_function_asm()
if __name__ == '__main__':
    carpeta= ls("/content/fotos")
    print(carpeta)
    tiempoc=[]
    tiempopy=[]
    tiempoasm=[]
    tiempofun=[]
    
    speeduppy=[]
    speedupfun=[]
    speedupc=[]
    for m in carpeta:
      img = Image.open(r"/content/fotos/%s" %str(m))
      img_data = img.getdata()
      #aqui estan los datos de c
      #limite_c = xy * 3
      limite_c=len(img_data)
      #se genera un arreglo donde saldran los valores de resultado
      salida_c = np.zeros((limite_c,1),dtype=np.float32)
      #aqui se almacena un arreglo dentro de la codificacion en C
      entrada_c=np.array(img_data) 
      entrada_c=entrada_c.flatten()

      print(entrada_c)
      print(limite_c)

      #######################################################################
                  #EXCLUSIVO DE PYTHON FUNCIONAL
      #######################################################################
      inicio_fun=time.perf_counter()
      image = cv2.imread('/content/fotos/%s' %str(m))
      #print(image)

      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      
      #print(i)
      #cv2.imshow('Original image RADIO',image)
      cv2.imwrite(f"/content/fotos/SALIDA en PYTHON de {str(m)}.png", gray)
      fin_fun=time.perf_counter()
      tiempofun.append(fin_fun-inicio_fun)

  #######################################################################
              #EXCLUSIVO DE LENGUAJE C
  #######################################################################

      # indicar la ruta de la shared library
      libfile = './GC_c.so'

      # cargar la shared library
      lib = ctypes.CDLL(libfile)

      # tipo de dato de los argumentos
      lib.GC.argtypes = [
          np.ctypeslib.ndpointer(dtype=np.float32),
          np.ctypeslib.ndpointer(dtype=np.float32),
          ctypes.c_int
          ]
      # funcion configurada

      funcion_c = lib.GC
      entrada_c=entrada_c.astype("float32")
      salida_c=salida_c.astype("float32")
      limite_c=int(limite_c)

      inicio_c = time.perf_counter()
      funcion_c(entrada_c,salida_c,limite_c)
      fin_c= time.perf_counter()

      tiempoc.append(fin_c - inicio_c)

      salida_c=salida_c.round()
      salida_c=salida_c.astype(int)
      #print(entrada_c)
      #print(salida_c[0:100])
      

    #######################################################################
                #EXCLUSIVO DE ENSAMBLADOR
    #######################################################################

      limite_asm=int(len(img_data))
      #se genera un arreglo donde saldran los valores de resultado
      salida_asm = np.zeros((limite_c,1),dtype=np.float32)
      #aqui se almacena un arreglo dentro de la codificacion en C
      entrada_asm=np.array(img_data,dtype=np.float32) 
      entrada_asm=entrada_asm.flatten()

      # indicar la ruta de la shared library
      libfiles = './grayscale_ASMO.so'

      # cargar la shared library
      libr = ctypes.CDLL(libfiles)

      # tipo de dato de los argumentos
      libr.GS_.argtypes = [
          np.ctypeslib.ndpointer(dtype=np.float32),
          np.ctypeslib.ndpointer(dtype=np.float32),
          ctypes.c_int
      ]

      
      funcion_asm = libr.GS_
      inicio_asm = time.perf_counter()
      funcion_asm(entrada_asm,salida_asm,limite_asm)
      fin_asm = time.perf_counter()

      tiempoasm.append(fin_asm - inicio_asm)

      salida_asm=salida_asm.round()
      salida_asm=salida_asm.astype(int)
      #print(entrada_asm)
      #print(salida_asm[0:1000])

      lst=[]

      #######################################################################
                  #EXCLUSIVO DE PYTHON ITERATIVO
      #######################################################################
      inicio_py = time.perf_counter()
      for i in img_data:
        #lst.append(i[0]*0.299+i[1]*0.587+i[2]*0.114) ### Rec. 609-7 weights
        try:
          lst.append(i[0]*0.2125+i[1]*0.7174+i[2]*0.0721) ### Rec. 709-6 weights
        except:
          pass
      fin_py = time.perf_counter()

      tiempopy.append(fin_py - inicio_py)


      speedupfun.append((fin_fun-inicio_fun)/(fin_asm - inicio_asm))
      speeduppy.append((fin_py - inicio_py)/(fin_asm - inicio_asm))
      speedupc.append((fin_c - inicio_c)/(fin_asm - inicio_asm))

      #######################################################################
                  #MUESTRA DE IMAGENES
      #######################################################################



      new_img = Image.new("L", img.size)
      new_img.putdata(lst)
      new_img.save(f"SALIDA en python de %s" %str(m))
      new_img.show()

      new_img_c =Image.new("L", img.size)
      new_img_c.putdata(salida_c)
      new_img_c.save("SALIDA EN C de %s" %str(m))
      new_img_c.show()

      new_img_asm =Image.new("L", img.size)
      new_img_asm.putdata(salida_asm)
      new_img_asm.save("SALIDA EN ASM de %s" %str(m))
      new_img_asm.show()

      print(OKBLUE+f"la funcion en python iterativo de {str(m)} es de {fin_py-inicio_py} segundos")
      print(OKCYAN+f"la funcion en python funcional de {str(m)} es de {fin_fun-inicio_fun} segundos")
      print(OKGREEN+f"la funcion en c de {str(m)} es de {fin_c-inicio_c} segundos")
      print(YELLOW+f"la funcion en asm de {str(m)} es de {fin_asm-inicio_asm} segundos")

    #print(purple+"#######################################################################")
    print(OKBLUE+f"el tiempo total de python iterativo es de {sum(tiempopy)} segundos")
    print(OKCYAN+f"el tiempo total de python funcional es de {sum(tiempofun)} segundos")
    print(OKGREEN+f"el tiempo total de c es de {sum(tiempoc)} segundos")
    print(YELLOW+f"el tiempo total de asm es de {sum(tiempoasm)} segundos")
    tiempos_sumados=[sum(tiempopy),sum(tiempofun),sum(tiempoc),sum(tiempoasm)]
    tiempos_MENORES=[sum(tiempofun),sum(tiempoc),sum(tiempoasm)]
    index = carpeta
    index2 = range(len(tiempoasm))
    index3 = range(len(tiempoc))
    plt.xlabel('tamagno')
    plt.ylabel('tiempo')
    plt.xticks(rotation=90)
    plt.title("TIEMPOS DE PYTHON/ASM Y C",fontdict={'color' : 'darkblue','size': 10})
    plt.plot(index, tiempopy, 'r-o', label="py iterativo")
    plt.plot(index, tiempoasm, 'g-o', label="asm")
    plt.plot(index, tiempoc, 'b-o', label="c")
    plt.plot(index, tiempofun, 'm-o', label="py funcional")
    plt.legend()
    plt.tight_layout()
    plt.savefig("RESULTADO DE TIEMPO.png")
    #plt.show
    plt.close()
    plt.xlabel('tamagno')
    plt.ylabel('speedup')
    plt.xticks(rotation=90)
    plt.title("SPEEDUP",fontdict={'color' : 'darkblue','size': 10})
    plt.plot(index, speeduppy, 'b-+', label="python")
    plt.plot(index, speedupc, 'g-+', label="c")
    plt.plot(index, speedupfun, 'm-+', label="py funcional")
    plt.legend()
    plt.tight_layout()
    plt.savefig("SPEEDUP.png")
    #plt.show
    plt.close()
    index = ["PY ITERATIVO","PY FUNCIONAL", "C", "ASM"]
    plt.xlabel('metodo')
    #plt.xticks(rotation=90)
    plt.ylabel('tiempo')
    plt.title(f"TIEMPOS TOTALES ",fontdict={'color' : 'darkblue','size': 6})
    plt.bar( index, tiempos_sumados, color='c')
    #plt.plot(index, tiempos_sumados, 'c-o', label="funcional")
    #plt.legend()
    plt.tight_layout()
    plt.savefig(f"./TIEMPOS TOTALES.png")
    plt.close()
    index = ["PY FUNCIONAL", "C", "ASM"]
    plt.xlabel('metodo')
    #plt.xticks(rotation=90)
    plt.ylabel('tiempo')
    plt.title(f"TIEMPOS TOTALES ",fontdict={'color' : 'darkblue','size': 6})
    plt.bar( index, tiempos_MENORES, color='m')
    #plt.plot(index, tiempos_sumados, 'c-o', label="funcional")
    #plt.legend()
    plt.tight_layout()
    plt.savefig(f"./TIEMPOS MENORES.png")

